# 🚀 Complete GPU Server - Image + Voice API (FIXED)
## ✅ Fixes ALL Issues: Kokoro Import, SDXL Files, diffusers

### Critical Fixes Applied:
1. ✅ **Kokoro TTS**: Proper installation & KPipeline usage
2. ✅ **SDXL Images**: Absolute paths, file verification, proper saving
3. ✅ **diffusers**: Correct imports (DiffusionPipeline)

### Instructions:
1. **Runtime → Change runtime type → GPU (T4)**
2. Run cells 1-6 in order
3. Add your ngrok token in cell 7
4. Run cell 7 and 8 to start server
5. Copy the ngrok URL
6. Update your local `.env` file

In [ ]:
# 📦 STEP 1: Install Dependencies (WITH PROPER VERSIONS)
print("\n" + "="*60)
print("📦 Installing Dependencies...")
print("="*60 + "\n")

# Install PyTorch with CUDA support
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Install diffusers with compatible version
!pip install -q "diffusers>=0.24.0" transformers accelerate safetensors

# Install Flask and ngrok
!pip install -q flask flask-cors pyngrok

# Install image/audio libraries
!pip install -q pillow numpy soundfile scipy

# ✅ CRITICAL: Install Kokoro TTS BEFORE any imports
print("\n🎤 Installing Kokoro TTS...")
!pip install -q "kokoro-onnx>=0.1.0"

print("\n✅ All dependencies installed!")
print("   ⚠️  DO NOT restart runtime before running next cells!")

In [ ]:
# 🔧 STEP 2: Import Libraries and Check GPU
print("\n" + "="*60)
print("🔧 Importing libraries and checking environment...")
print("="*60 + "\n")

import torch
import io
import base64
import os
from PIL import Image
from flask import Flask, request, jsonify
from flask_cors import CORS
import numpy as np
from pathlib import Path

# Check GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🎮 Device: {device}")
if device == "cuda":
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("   ⚠️  WARNING: No GPU! Change Runtime → Change runtime type → GPU")

# Create output directories with absolute paths
IMAGES_DIR = "/content/generated_images"
AUDIO_DIR = "/content/generated_audio"
os.makedirs(IMAGES_DIR, exist_ok=True)
os.makedirs(AUDIO_DIR, exist_ok=True)

print(f"\n📁 Working directories:")
print(f"   Images: {IMAGES_DIR}")
print(f"   Audio: {AUDIO_DIR}")
print("\n✅ Environment ready!")

In [ ]:
# 🎨 STEP 3: Load SDXL-Turbo (FIXED IMPORT)
print("\n" + "="*60)
print("🎨 Loading SDXL-Turbo for image generation...")
print("="*60 + "\n")

from diffusers import DiffusionPipeline

# Load SDXL-Turbo with correct pipeline class
print("📥 Downloading model (first time only, ~7GB)...")
sdxl_pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/sdxl-turbo",
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    variant="fp16" if device == "cuda" else None,
    use_safetensors=True
)

sdxl_pipe = sdxl_pipe.to(device)

# Enable optimizations for T4 GPU
if device == "cuda":
    try:
        sdxl_pipe.enable_xformers_memory_efficient_attention()
        print("   ✅ xFormers memory optimization enabled")
    except:
        print("   ℹ️  xFormers not available, using default attention")
    
    # Enable VAE slicing for lower VRAM
    sdxl_pipe.enable_vae_slicing()
    print("   ✅ VAE slicing enabled (lower VRAM usage)")

print("\n✅ SDXL-Turbo loaded successfully!")
print(f"   Model: stabilityai/sdxl-turbo")
print(f"   Device: {device}")
print(f"   Resolution: 1024x1024")
print(f"   Speed: 1-4 inference steps")

In [ ]:
# 🎤 STEP 4: Setup Kokoro TTS (FIXED - KPipeline)
print("\n" + "="*60)
print("🎤 Setting up Kokoro TTS...")
print("="*60 + "\n")

# ✅ CRITICAL FIX: Import kokoro AFTER installation in Step 1
try:
    from kokoro import KPipeline
    import soundfile as sf
    KOKORO_AVAILABLE = True
    print("✅ Kokoro module imported successfully!")
except ImportError as e:
    print(f"❌ ERROR: Cannot import Kokoro: {e}")
    print("   Please restart runtime and run cells again from the beginning.")
    KOKORO_AVAILABLE = False

# Global TTS pipeline
tts_pipeline = None

def load_tts_model():
    """Load Kokoro TTS pipeline (lazy loading)"""
    global tts_pipeline
    
    if not KOKORO_AVAILABLE:
        raise RuntimeError("Kokoro is not available. Restart runtime and re-run all cells.")
    
    if tts_pipeline is None:
        print("   🎤 Loading Kokoro TTS pipeline...")
        # Use 'a' for American English (supports most voices)
        tts_pipeline = KPipeline(lang_code='a')
        print("   ✅ Kokoro pipeline loaded!")
    
    return tts_pipeline

# Voice mapping: user-friendly → Kokoro voice IDs
VOICE_MAP = {
    'aria': 'af_sarah',
    'guy': 'am_adam',
    'jenny': 'af_nicole',
    'christopher': 'am_michael',
    'sara': 'af_bella',
    'sarah': 'af_sarah',
    'roger': 'am_adam',
    'nancy': 'af_jessica',
    'andrew': 'am_adam',
}

if KOKORO_AVAILABLE:
    print(f"\n✅ Kokoro TTS ready!")
    print(f"   Available voices: {len(VOICE_MAP)}")
    print(f"   Sample rate: 24kHz")
    print(f"   Quality: Professional studio")
else:
    print("\n❌ Kokoro TTS NOT available!")
    print("   Audio generation will fail.")

In [ ]:
# 🔌 STEP 5: Create Flask API Server (FIXED)
print("\n" + "="*60)
print("🔌 Creating Flask API server...")
print("="*60 + "\n")

app = Flask(__name__)
CORS(app)

# ═══════════════════════════════════════════════════════════════
# IMAGE GENERATION - FIXED FILE SAVING
# ═══════════════════════════════════════════════════════════════

@app.route('/generate_image', methods=['POST'])
def generate_image():
    """Generate image with SDXL-Turbo (FIXED)"""
    try:
        data = request.json
        prompt = data.get('prompt', '')
        scene_id = data.get('scene_id', 0)
        
        if not prompt:
            return jsonify({'error': 'Prompt is required'}), 400
        
        print(f"\n🎨 Generating image for scene {scene_id}...")
        print(f"   Prompt: {prompt[:80]}...")
        
        # Generate with SDXL-Turbo
        image = sdxl_pipe(
            prompt=prompt,
            num_inference_steps=2,
            guidance_scale=0.0,
            height=1024,
            width=1024
        ).images[0]
        
        # ✅ FIX: Save to absolute path and verify
        import time
        timestamp = int(time.time() * 1000)
        filename = f"scene_{scene_id:03d}_{timestamp}.png"
        save_path = os.path.join(IMAGES_DIR, filename)
        
        print(f"   💾 Saving to: {save_path}")
        image.save(save_path, format='PNG')
        
        # ✅ VERIFY file exists
        if not os.path.exists(save_path):
            raise RuntimeError(f"Failed to save image! Path: {save_path}")
        
        file_size = os.path.getsize(save_path) / 1024  # KB
        print(f"   ✅ Saved successfully! Size: {file_size:.1f} KB")
        
        # Convert to base64 for transmission
        img_byte_arr = io.BytesIO()
        image.save(img_byte_arr, format='PNG')
        img_byte_arr.seek(0)
        img_base64 = base64.b64encode(img_byte_arr.getvalue()).decode('utf-8')
        
        return jsonify({
            'success': True,
            'image': img_base64,
            'format': 'png',
            'filename': filename,
            'path': save_path,
            'size_kb': file_size
        })
    
    except Exception as e:
        print(f"   ❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()
        return jsonify({'error': str(e)}), 500

# ═══════════════════════════════════════════════════════════════
# VOICE GENERATION - FIXED KOKORO IMPORT
# ═══════════════════════════════════════════════════════════════

@app.route('/generate_audio', methods=['POST'])
def generate_audio():
    """Generate audio with Kokoro TTS (FIXED)"""
    try:
        data = request.json
        text = data.get('text', '')
        voice = data.get('voice', 'aria')
        speed = float(data.get('speed', 1.0))
        
        if not text:
            return jsonify({'error': 'Text is required'}), 400
        
        if not KOKORO_AVAILABLE:
            return jsonify({'error': 'Kokoro TTS is not available. Restart runtime.'}), 500
        
        # Map voice
        kokoro_voice = VOICE_MAP.get(voice.lower(), 'af_sarah')
        
        print(f"\n🎤 Generating audio: {voice} → {kokoro_voice}")
        print(f"   Text: {text[:80]}...")
        print(f"   Speed: {speed}x")
        
        # ✅ Load pipeline
        pipeline = load_tts_model()
        
        # ✅ Generate audio chunks
        if len(text) > 1000:
            print(f"   📝 Splitting into chunks (text is long)...")
            chunks = split_text_smart(text, max_chars=800)
            audio_segments = []
            
            for i, chunk in enumerate(chunks):
                print(f"   Processing chunk {i+1}/{len(chunks)}...")
                results = list(pipeline(chunk, voice=kokoro_voice, speed=speed))
                
                if not results or not hasattr(results[0], 'audio'):
                    raise RuntimeError(f"Chunk {i+1} failed: no audio returned")
                
                audio_tensor = results[0].audio.cpu().numpy()
                audio_segments.append(audio_tensor)
            
            # Concatenate
            audio = np.concatenate(audio_segments)
        else:
            # Short text - generate directly
            results = list(pipeline(text, voice=kokoro_voice, speed=speed))
            
            if not results or not hasattr(results[0], 'audio'):
                raise RuntimeError("Kokoro returned no audio")
            
            audio = results[0].audio.cpu().numpy()
        
        # ✅ Save audio to file
        import time
        timestamp = int(time.time() * 1000)
        filename = f"audio_{timestamp}.wav"
        save_path = os.path.join(AUDIO_DIR, filename)
        
        sample_rate = 24000  # Kokoro sample rate
        sf.write(save_path, audio, sample_rate)
        
        # ✅ VERIFY
        if not os.path.exists(save_path):
            raise RuntimeError(f"Failed to save audio! Path: {save_path}")
        
        # Convert to base64
        audio_byte_arr = io.BytesIO()
        sf.write(audio_byte_arr, audio, sample_rate, format='WAV')
        audio_byte_arr.seek(0)
        audio_base64 = base64.b64encode(audio_byte_arr.getvalue()).decode('utf-8')
        
        duration = len(audio) / sample_rate
        print(f"   ✅ Audio generated! Duration: {duration:.1f}s")
        
        return jsonify({
            'success': True,
            'audio': audio_base64,
            'format': 'wav',
            'duration': duration,
            'sample_rate': sample_rate,
            'filename': filename,
            'path': save_path
        })
    
    except Exception as e:
        print(f"   ❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()
        return jsonify({'error': str(e)}), 500

# ═══════════════════════════════════════════════════════════════
# HEALTH CHECK
# ═══════════════════════════════════════════════════════════════

@app.route('/health', methods=['GET'])
def health():
    """Health check endpoint"""
    return jsonify({
        'status': 'ok',
        'device': device,
        'gpu': torch.cuda.get_device_name(0) if device == 'cuda' else 'none',
        'services': {
            'sdxl_turbo': 'ready',
            'kokoro_tts': 'ready' if KOKORO_AVAILABLE else 'unavailable'
        },
        'paths': {
            'images': IMAGES_DIR,
            'audio': AUDIO_DIR
        }
    })

# ═══════════════════════════════════════════════════════════════
# HELPER
# ═══════════════════════════════════════════════════════════════

def split_text_smart(text, max_chars=800):
    """Split text at sentence boundaries"""
    sentences = text.replace('!', '.').replace('?', '.').split('.')
    sentences = [s.strip() + '.' for s in sentences if s.strip()]
    
    chunks = []
    current = ""
    
    for sent in sentences:
        if len(current) + len(sent) <= max_chars:
            current += " " + sent
        else:
            if current:
                chunks.append(current.strip())
            current = sent
    
    if current:
        chunks.append(current.strip())
    
    return chunks if chunks else [text]

print("\n✅ Flask API server created!")
print("   Endpoints:")
print("   - POST /generate_image")
print("   - POST /generate_audio")
print("   - GET  /health")

In [ ]:
# 🌐 STEP 6: Setup ngrok
print("\n" + "="*60)
print("🌐 ngrok Setup")
print("="*60 + "\n")

from pyngrok import ngrok

print("📝 Instructions:")
print("   1. Get free ngrok token: https://dashboard.ngrok.com/get-started/your-authtoken")
print("   2. Edit the NGROK_AUTH_TOKEN variable below")
print("   3. Run this cell")
print("   4. Then run STEP 7 to start the server")
print("\n" + "="*60)

In [ ]:
# 🔑 STEP 7: Set ngrok Token (EDIT THIS!)

# ⬇️ REPLACE WITH YOUR ACTUAL TOKEN ⬇️
NGROK_AUTH_TOKEN = "YOUR_NGROK_TOKEN_HERE"
# ⬆️ REPLACE WITH YOUR ACTUAL TOKEN ⬆️

if NGROK_AUTH_TOKEN == "YOUR_NGROK_TOKEN_HERE":
    print("❌ ERROR: You must set your ngrok token!")
    print("\n📝 Steps:")
    print("   1. Get token: https://dashboard.ngrok.com/get-started/your-authtoken")
    print("   2. Edit NGROK_AUTH_TOKEN in the cell above")
    print("   3. Run this cell again")
else:
    from pyngrok import ngrok
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    print("✅ ngrok token configured!")
    print("\n➡️  Now run STEP 8 to start the server.")

In [ ]:
# 🚀 STEP 8: START SERVER!
print("\n" + "="*60)
print("🚀 STARTING GPU SERVER")
print("="*60 + "\n")

from pyngrok import ngrok

# Start ngrok tunnel
public_url = ngrok.connect(5000, bind_tls=True)

print("\n" + "="*60)
print("✅ GPU SERVER IS RUNNING!")
print("="*60)
print(f"\n🌐 PUBLIC URL:")
print(f"   {public_url}")
print(f"\n" + "="*60)
print("📋 SETUP YOUR LOCAL APP:")
print("="*60)
print(f"\n1. Create/edit .env file in your project:")
print(f"\n   SDXL_API_URL={public_url}/generate_image")
print(f"   KOKORO_API_URL={public_url}/generate_audio")
print(f"\n2. Restart your local app")
print(f"\n3. Generate videos!")
print(f"\n" + "="*60)
print(f"🧪 TEST ENDPOINTS:")
print("="*60)
print(f"\n   curl {public_url}/health")
print(f"\n" + "="*60)
print("⚠️  Keep this notebook running!")
print("   Closing it will stop the server.")
print("="*60 + "\n")

# Start Flask (blocking)
app.run(port=5000, debug=False, use_reloader=False)